### Step 1: Import packages

In [102]:
from google.cloud import bigquery
from google.cloud import bigquery_storage
import pandas as pd
import numpy as np
import pingouin as pg
import datetime as dt
from datetime import datetime, timedelta
import uuid
import scipy.stats
import warnings
warnings.filterwarnings(action="ignore")

### Step 2.1: Define some input parameters

In [71]:
exp_length = 28
sb_window_size = 3
# Define the list of KPIs
col_list = [
    'actual_df_paid_by_customer', 'gfv_local', 'gmv_local', 'commission_local', 'joker_vendor_fee_local', # Customer KPIs (1)
    'sof_local', 'service_fee_local', 'revenue_local', 'delivery_costs_local', 'gross_profit_local', # Customer KPIs (2)
    'dps_mean_delay', 'delivery_distance_m', 'actual_DT' # Logistics KPIs
]

### Step 2.2: Instantiate a BQ client and run the SQL query that pulls the historical data

In [3]:
client = bigquery.Client(project="logistics-data-staging-flat")
bqstorage_client = bigquery_storage.BigQueryReadClient()

with open("sql_queries.sql", mode="r", encoding="utf-8") as f:
    query = f.read()
    f.close()

client.query(query=query).result()

In [4]:
# Pull the data from the final table generated by the query
df = client.query("""SELECT * FROM `dh-logistics-product-ops.pricing.ab_test_individual_orders_augmented_randomization_algo_analysis`""").result().to_dataframe(bqstorage_client=bqstorage_client, progress_bar_type="tqdm")

Downloading: 100%|██████████| 1422904/1422904 [01:07<00:00, 21035.21rows/s]


### Step 3.1: Define a list of dictionaries containing the entity IDs, ASA IDs, and zone names that will be used in the analysis

In [5]:
entity_asa_zone_dict = [
    # SG
    {"entity_id": "FP_SG", "asa_id": 559, "zone_names": ["Bukitpanjang", "Jurongwest", "Woodlands"], "zone_group_identifier": "zg_1"},
    {"entity_id": "FP_SG", "asa_id": 560, "zone_names": ["Far_east", "Jurong east"], "zone_group_identifier": "zg_2"},

    # HK
    {"entity_id": "FP_HK", "asa_id": 402, "zone_names": ["To kwa wan rider", "Kowloon city rider", "Lai chi kok rider"], "zone_group_identifier": "zg_3"},
    {"entity_id": "FP_HK", "asa_id": 406, "zone_names": ["Ma liu shui rider", "Kwai chung rider", "Sai kung rider", "Sheung shui rider", "Tai po rider", "Tai wai rider", "Tin shui wai rider", "Tsing yi rider", "Tsuen wan rider", "Tuen mun rider", "Tun chung rider", "Yuen long rider"], "zone_group_identifier": "zg_4"},
    {"entity_id": "FP_HK", "asa_id": 398, "zone_names": ["Admiralty cwb rider", "Happy valley cwb rider", "Kennedy town rider", "Quarry bay rider"], "zone_group_identifier": "zg_5"},

    # PH
    {"entity_id": "FP_PH", "asa_id": 496, "zone_names": ["South alabang atc", "Paranaque", "North Ias pinas", "North alabang atc", "Bf homes"], "zone_group_identifier": "zg_6"},
    {"entity_id": "FP_PH", "asa_id": 525, "zone_names": ["Bacoor north", "Tagaytay", "Dasmarinas", "Imus"], "zone_group_identifier": "zg_7"},
    {"entity_id": "FP_PH", "asa_id": 528, "zone_names": ["Antipolo north", "Malabon", "Sjdm", "Valenzuela"], "zone_group_identifier": "zg_8"},
    {"entity_id": "FP_PH", "asa_id": 508, "zone_names": ["Makati", "Pasay"], "zone_group_identifier": "zg_9"}
]

### Step 3.2: Create a new data frame with the combinations stipulated in the dictionary above

In [6]:
df_reduced = []
for i in entity_asa_zone_dict:
    df_iter = df[(df["entity_id"] == i["entity_id"]) & (df["asa_id"] == i["asa_id"]) & (df["zone_name"].isin(i["zone_names"]))]
    df_iter["zone_group_identifier"] = i["zone_group_identifier"]
    df_reduced.append(df_iter)

# Convert df_reduced to a dataframe
df_reduced = pd.concat(df_reduced)

# Add a new field to df_reduced showing a different format of "dps_sessionid_created_at_utc". We want to display the format followed by DPS, which is "%Y-%m-%dT%H:%M:%SZ"
df_reduced["dps_sessionid_created_at_utc_formatted"] = df_reduced["dps_sessionid_created_at_utc"]\
    .apply(lambda x: pd.to_datetime(dt.datetime.strftime(x, "%Y-%m-%dT%H:%M:%SZ")))

df_reduced.reset_index(drop=True, inplace=True)

In [7]:
# The shell script that runs the randomization algorithm needs the starting time of the experiment as one of its input
# We define that as the minimum dps_session_start_timestamp per zone_group_identifier
df_min_max_dps_session_start_ts = df_reduced.groupby(["entity_id", "zone_group_identifier"])["dps_sessionid_created_at_utc_formatted"]\
    .agg(["min", "max"])\
    .reset_index()\
    .rename(columns={"min": "min_dps_session_start_ts", "max": "max_dps_session_start_ts"})

### Step 4.1: Create a function that takes the zone_group_identifier and creates a CSV file called input_{zg_identifier}. This file contains the details necessary to run the randomization algorithm

In [31]:
def input_csv_func(zg_identifier):
    df_stg = df_reduced[df_reduced["zone_group_identifier"] == zg_identifier][["platform_order_code", "zone_id", "dps_sessionid_created_at_utc_formatted"]]\
        .sort_values("dps_sessionid_created_at_utc_formatted")\
        .reset_index(drop=True)
    df_stg["dps_sessionid_created_at_utc_formatted"] = df_stg["dps_sessionid_created_at_utc_formatted"].apply(lambda x: str(x))
    df_stg.to_csv(f"input.csv", index=False, header=False, date_format="str")

# Invoke the function that creates the input file. Keep in mind that this overwrites the already existing input.csv file
input_csv_func(zg_identifier="zg_1")

### Step 4.2: Convert the CSV file to UNIX format and run the variant allocation function

In [32]:
%%script "C:/Program Files/Git/bin/bash.exe"
dos2unix input.csv

dos2unix: converting file input.csv to Unix format...


In [33]:
%%script "C:/Program Files/Git/bin/bash.exe"
./run-allocation.sh -w 3.0 -v 3 -t 2023-01-01T00:48:04Z -k 28115 -s DB0720FD-326E-407F-9EA2-512BF8154DDE

key: 28115
salt: DB0720FD-326E-407F-9EA2-512BF8154DDE
Switchback parameters are valid, starting experiment..
Allocation is complete. Results are available in output.csv file


### Step 5: Create a function that gives a random UUID to each time interval. Note: This part will be removed once the UUID functionality is incorporated in the JS function

In [169]:
def hr_interval_date_func_random(zg_id, test_length, sb_interval, zone_name_list: list):
    min_timestamp_zg_id = df_min_max_dps_session_start_ts[df_min_max_dps_session_start_ts["zone_group_identifier"] == zg_id].reset_index()["min_dps_session_start_ts"][0]
    num_time_units = int((24 / sb_interval) * test_length)

    # Create an array of timestamps separated by the switchback window size
    df_mapping = [min_timestamp_zg_id] # Decalre teh df_mapping variable as a list with the first value being min_timestamp_zg_id
    timestamp_iter = min_timestamp_zg_id # Initialize the timestamp_iter with min_timestamp_zg_id
    for i in range(1, num_time_units):
        df_mapping.append(timestamp_iter + timedelta(hours = 3))
        timestamp_iter = timestamp_iter + timedelta(hours=3) # Update the 
    df_mapping = pd.DataFrame(df_mapping, columns=["dps_session_created_at"]) # Convert the list to a data frame

    # Create new columns
    df_mapping["dps_session_created_date"] = df_mapping["dps_session_created_at"].apply(lambda x: pd.to_datetime(x.date()))
    df_mapping["dps_session_created_at_interval"] = pd.cut(df_mapping["dps_session_created_at"], bins=num_time_units, right=False)
    df_mapping["common_key"] = 0

    # Create a new data frame containing the zones in the zone group ID
    df_zone_id = pd.DataFrame({"zone_name": zone_name_list, "common_key": 0})
    df_mapping = pd.merge(left=df_mapping, right=df_zone_id, how="outer", on="common_key")
    df_mapping.drop("common_key", axis=1, inplace=True)
    
    rnd_id_list = [] # Create the full list that the rng.choice would choose from
    for i in range(1, len(df_mapping) + 1):
        rnd_id_list.append(uuid.uuid4())

    rng = np.random.default_rng()
    df_mapping['time_zone_unit_id'] = rng.choice(rnd_id_list, replace = False, axis = 0, size = len(df_mapping))
    return df_mapping

In [170]:
hr_interval_date_func_random(zg_id="zg_1", test_length=exp_length, sb_interval=sb_window_size, zone_name_list=df_analysis["zone_name"].unique()).head(10)

,dps_session_created_at,dps_session_created_date,dps_session_created_at_interval,zone_name,time_zone_unit_id
0,2023-01-01 00:48:04+00:00,2023-01-01,"[2023-01-01 00:48:04, 2023-01-01 03:47:15.7857...",Jurongwest,84ec420d-7ff7-4b33-882c-efc183b1cbb9
1,2023-01-01 00:48:04+00:00,2023-01-01,"[2023-01-01 00:48:04, 2023-01-01 03:47:15.7857...",Woodlands,62da6476-0ae3-495d-843d-f10c50fe4455
2,2023-01-01 00:48:04+00:00,2023-01-01,"[2023-01-01 00:48:04, 2023-01-01 03:47:15.7857...",Bukitpanjang,b920b1e5-7816-493f-a672-e62eb5caa87c
3,2023-01-01 03:48:04+00:00,2023-01-01,"[2023-01-01 03:47:15.785714176, 2023-01-01 06:...",Jurongwest,c49d4272-c143-497d-a329-0a921e9b588c
4,2023-01-01 03:48:04+00:00,2023-01-01,"[2023-01-01 03:47:15.785714176, 2023-01-01 06:...",Woodlands,fa791024-3d3d-4cf4-8b20-6e2a01d26661
5,2023-01-01 03:48:04+00:00,2023-01-01,"[2023-01-01 03:47:15.785714176, 2023-01-01 06:...",Bukitpanjang,3ec6d74d-f5aa-465e-9921-67d1497c37ec
6,2023-01-01 06:48:04+00:00,2023-01-01,"[2023-01-01 06:46:27.571428608, 2023-01-01 09:...",Jurongwest,0e5fa4b1-f5cc-4774-893a-63da6afde383
7,2023-01-01 06:48:04+00:00,2023-01-01,"[2023-01-01 06:46:27.571428608, 2023-01-01 09:...",Woodlands,d6593659-514c-4c8e-945e-1bad9a1526ec
8,2023-01-01 06:48:04+00:00,2023-01-01,"[2023-01-01 06:46:27.571428608, 2023-01-01 09:...",Bukitpanjang,eab84d22-d9db-4d7e-a7d3-06a958790631
9,2023-01-01 09:48:04+00:00,2023-01-01,"[2023-01-01 09:45:39.357142784, 2023-01-01 12:...",Jurongwest,60a77859-4a8b-42ef-ac64-a1c9684e44ba


### Step 6.1: After the output.csv file is created, retrieve the variants from the output.csv file and join them to df_reduced

In [130]:
df_variants = pd.read_csv("output.csv")
df_analysis = df_reduced[df_reduced["zone_group_identifier"] == "zg_1"].copy() # Create a copy of df_reduced just for the zg_id being analysed
df_analysis = pd.merge(left=df_analysis, right=df_variants, how="left", left_on="platform_order_code", right_on="OrderID")
df_analysis.drop("OrderID", axis=1, inplace=True)

### Step 6.2: Do some extra operations on df_analysis

In [131]:
# Add a column indicating the week number
df_analysis["dps_session_created_date"] = df_analysis["dps_sessionid_created_at_utc_formatted"].apply(lambda x: pd.to_datetime(datetime.date(x)))

# Change the KPI columns to numeric
df_analysis[col_list] = df_analysis[col_list].apply(lambda x: pd.to_numeric(x))

# Create a conditions list
start_date = df_analysis["created_date_local"].min()
conditions = [
    (df_analysis["created_date_local"] >= start_date) & (df_analysis["created_date_local"] <= start_date + timedelta(6)),
    (df_analysis["created_date_local"] >= start_date + timedelta(7)) & (df_analysis["created_date_local"] <= start_date + timedelta(13)),
    (df_analysis["created_date_local"] >= start_date + timedelta(14)) & (df_analysis["created_date_local"] <= start_date + timedelta(20)),
    (df_analysis["created_date_local"] >= start_date + timedelta(21)) & (df_analysis["created_date_local"] <= start_date + timedelta(27)),
]

df_analysis["week_num"] = np.select(condlist=conditions, choicelist=["week_1", "week_2", "week_3", "week_4"])

# Create the data frame containing the random UUIDs for each time interval
df_mapping = hr_interval_date_func_random(zg_id="zg_1", test_length=exp_length, sb_interval=sb_window_size, zone_name_list=df_analysis["zone_name"].unique())

# Create a function that returns the right hr_interval from df_mapping for any given number
def check_right_interval(num, col):
    for i in col:
        if num in i:
            return i
        else:
            pass

# Get the right interval using the "check_right_interval" function
df_analysis['dps_session_created_at_interval'] = df_analysis\
    .apply(lambda x: check_right_interval(x['dps_sessionid_created_at_utc_formatted'], df_mapping['dps_session_created_at_interval']), axis = 1)

# Filter df_analysis based on exp_length
df_analysis = df_analysis[df_analysis["day_num"] <= exp_length]

# Merge the random UUIDs with df_analysis. Note: This part will be removed once the UUID functionality is incorporated in the JS function
df_analysis = pd.merge(left = df_analysis, right = df_mapping, how = 'left', on = ["dps_session_created_at_interval", "zone_name", "dps_session_created_date"])

In [126]:
# Calculate the "total" metrics and rename the column label to "df_per_order_metrics"
df_analysis_tot = round(df_analysis.groupby(["time_zone_unit_id", "Variant"])[col_list[:-3]].sum(), 2)
df_analysis_tot['order_count'] = df_analysis.groupby(["time_zone_unit_id", "Variant"])['platform_order_code'].nunique()
df_analysis_tot = df_analysis_tot.rename_axis(['df_tot_metrics'], axis = 1)


# Calculate the "total" metrics and rename the column label to "df_per_order_metrics"
df_analysis_per_order_cust_kpis = df_analysis_tot.copy()

for iter_col in df_analysis_per_order_cust_kpis.columns[:-1]:
    df_analysis_per_order_cust_kpis[iter_col] = round(df_analysis_per_order_cust_kpis[iter_col] / df_analysis_per_order_cust_kpis['order_count'], 4)

df_analysis_per_order_log_kpis = round(df_analysis.groupby(["time_zone_unit_id", "Variant"])[col_list[-3:]].mean(), 2) 
df_analysis_per_order = pd.concat([df_analysis_per_order_cust_kpis, df_analysis_per_order_log_kpis], axis = 1)
df_analysis_per_order = df_analysis_per_order.rename_axis(['df_per_order_metrics'], axis = 1)

# Reset the indices of the 
df_analysis_tot = df_analysis_tot.reset_index()
df_analysis_per_order = df_analysis_per_order.reset_index()

In [139]:
pg.welch_anova(dv="actual_df_paid_by_customer", between="Variant", data=df_analysis_tot)["p-unc"].iloc[0].round(4)

0.2997

In [90]:
 # Calculate the p-values using the Welch's t-test
pval_dict = {}
df_pval_final_list = []

for iter_col in df_analysis_per_order.columns[2:]:
    anova_pval_tot = pg.welch_anova(dv=iter_col, between="Variant", data=df_analysis_tot)["p-unc"].iloc[0].round(4)
    anova_pval_per_order = pg.welch_anova(dv=iter_col, between="Variant", data=df_analysis_per_order)["p-unc"].iloc[0].round(4)

df_pval_final_list.append(pval_dict)

# Convert the p-values to a data frame
df_pval_final_tbl = pd.DataFrame(df_pval_final_list)\
    .assign(entity_id = entity_id, zone_name = zone_name)\
    .set_index(['entity_id', 'zone_name'])

df_per_order_metrics,time_zone_unit_id,Variant,actual_df_paid_by_customer,revenue_local,delivery_costs_local,gross_profit_local,order_count,delivery_distance_m,actual_DT
0,001d931c-f693-4c55-930f-c094af3487cf,Variant1,10.9900,22.7800,14.5500,8.2300,1,8085.00,83.68
1,0161361c-c00b-4b76-8f36-fffb4032a676,Variant1,4.6900,10.6500,7.5400,3.1100,1,2401.00,64.23
2,0164a2ca-c344-4000-b29e-b48e80e4b2a0,Variant1,3.8900,19.8600,7.7500,12.1100,2,3905.00,31.96
3,02071705-8063-49ea-84c3-6f18ae9a8973,Variant0,5.4400,26.0600,10.5800,15.4800,4,4850.00,34.51
4,02b51ad1-4af1-4c00-bb19-ced160309b5c,Variant0,6.2043,17.2186,10.9829,6.2357,7,3744.57,33.46
...,...,...,...,...,...,...,...,...,...
300,fc2cf71f-6f95-4765-832a-df5dec30bec4,Variant1,7.1900,15.0450,14.0200,1.0250,2,5396.50,55.92
301,fe847d6a-4e12-4141-b476-121816931032,Variant1,4.1900,14.1300,8.0700,6.0600,1,1780.00,25.55
302,fed2a0c2-cfdd-46b1-8e85-faa012966db0,Variant1,4.4900,12.4888,10.0025,2.4862,8,4184.50,27.46
303,ff0cf02b-e7af-43d1-a4ec-af0540b64f92,Variant2,6.7233,12.2733,9.8733,2.4000,3,5675.00,28.31
